<a href="https://colab.research.google.com/github/pnkjkpvt/SAFE_OCR/blob/master/MNIST_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyyaml h5py  # Required to save models in HDF5 format


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls /content/gdrive

'My Drive'


In [4]:
import os

In [5]:
!mkdir /content/gdrive/'My Drive'/SAFE_OCR/models/MNIST_Model

In [6]:
import os

import torch

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


from google.colab.patches import cv2_imshow


Using TensorFlow backend.


In [ ]:
batch_size = 128
num_classes = 10
epochs = 20

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

cv2_imshow(x_train[0])
print(x_train[0])

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
# Define a simple sequential model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 12s 199us/step - loss: 0.2710 - accuracy: 0.9152 - val_loss: 0.0587 - val_accuracy: 0.9813
Epoch 2/20
60000/60000 [==============================] - 5s 79us/step - loss: 0.0887 - accuracy: 0.9742 - val_loss: 0.0394 - val_accuracy: 0.9865
Epoch 3/20
60000/60000 [==============================] - 5s 81us/step - loss: 0.0641 - accuracy: 0.9808 - val_loss: 0.0367 - val_accuracy: 0.9880
Epoch 4/20
60000/60000 [==============================] - 5s 80us/step - loss: 0.0537 - accuracy: 0.9843 - val_loss: 0.0327 - val_accuracy: 0.9895
Epoch 5/20
60000/60000 [==============================] - 5s 79us/step - loss: 0.0456 - accuracy: 0.9867 - val_loss: 0.0291 - val_accuracy: 0.9908
Epoch 6/20
60000/60000 [==============================] - 5s 79us/step - loss: 0.0403 - accuracy: 0.9878 - val_loss: 0.0328 - val_accuracy: 0.9900
Epoch 7/20
60000/60000 [==============================] - 5s 79us/

---
**Saving Model**

---



In [10]:
from keras.models import model_from_json

In [16]:
file_path = F"/content/gdrive/My Drive/SAFE_OCR/models/MNIST_Model/"
model_json = model.to_json()
config_json = str(model.get_config())
with open(file_path+"model.json", "w") as json_file:
    json_file.write(model_json)
with open(file_path+"config.json","w") as json_file:
    json_file.write(config_json)
# serialize weights to HDF5
model.save_weights(file_path+"model.h5")

print("Saved model to disk")

Saved model to disk


In [17]:
##Loading saved models

json_file = open(file_path+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(file_path+"model.h5")
print("Loaded model from disk")

Loaded model from disk


In [18]:
# Create a basic model instance
loaded_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Evaluate the model
score = loaded_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.028703007446681385
Test accuracy: 0.9922000169754028
